<a href="https://colab.research.google.com/github/kimhalyn/OPEN-API/blob/main/el_code%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리
# colab에 xmltodict 라이브러리가 기본 설치되지 않으므로 별도 설치(외부 라이브러리)
!pip install xmltodict 
from urllib.request import urlopen, Request
from urllib.parse import urlencode, unquote, quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests, xmltodict, json
from xml.etree.ElementTree import Element, SubElement, ElementTree
import xml.etree.ElementTree as ET
import pickle
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 전체 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')
damcode = data.iloc[:, 1] #인덱스
print(damcode)

In [ ]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from time import sleep
import csv

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/wal/list'
key = 'tMF2MGGUmsNNPf15Nr91%2BUA4YVQiWXVjLS7R0odxGSR0yfNjcHFkdRpCT6hXbILlBP1T%2BTnDJYz1kkDjiaIiQg=='
 
api_key = requests.utils.unquote(key) #디코딩

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 코드 조회
damcode = data.iloc[:, 1]
xmlrows = []
for code in damcode:

  Params = '?' + urlencode({quote_plus('ServiceKey') : api_key,
                                quote_plus('damcode') : code})
  
#  print(Params)
  
  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  sleep(0.5)
  print(res) 
  
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'
  print(xmlobj)
  
  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  xmlrows.append(rows)

print (xmlrows)


In [ ]:

filePath = '/content/drive/MyDrive/water_resource_csv/xmlrows.xml'

with open(filePath, 'wb') as lf:
    pickle.dump(xmlrows, lf)


with open(filePath, 'rb') as lf:
    readList = pickle.load(lf)


In [ ]:
rowList = []
nameList = []
columnList = []
xmlrowsLen = len(xmlrows)
target = 0
for i in range(0, xmlrowsLen):
    
   
    if len(xmlrows[i]) == 0:
       target = target + 1
      
    else:
        break
columns = xmlrows[target][0].find_all()
columnsLen = len(columns)
for i in range(0, columnsLen):
    nameList.append(columns[i].name)
nameList = ["Damcode"] + nameList


In [ ]:
xmlrows[0]

[]

In [ ]:
total = []
damcodeLen = len(damcode)


def el_code(a): 
    columnList = []
    eachrows = xmlrows[a]
    eachrowslen = len(eachrows)
    for i in range(0, eachrowslen):
        columns = eachrows[i].find_all()
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            eachColumn = columns[j].text 
            columnList.append(eachColumn)
        columnList = [damcode[a]] + columnList
        rowList.append(columnList)
        columnList = []
    return rowList



for a in range(0, damcodeLen):
    rowList=el_code(a)
    total.append(rowList)

result = pd.DataFrame(total[0], columns=nameList)

   

In [ ]:
result.to_csv("/content/drive/MyDrive/water_resource_csv/el_code.csv",encoding="utf-8-sig")

In [ ]:
result

,Damcode,obsrvtnm,walobsrvtcode
0,2011602,강정고령보상류,2011602
1,2011602,강정고령보하류,2011655
2,1007601,강천보상류,1007601
3,1007601,강천보하류,1005681
4,3012601,공주보상류,3012601
...,...,...,...
202,1006110,섬강교,9000250
203,1006110,횡성 수위감지,9900010
204,1006110,횡성교,8000093
205,1006110,횡성군(율동리),1006610
